In [ ]:
# .pbファイル読み込み
tf_model_path = './inception_v1_2016_08_28_frozen.pb'

with open(tf_model_path, 'rb') as f:
    serialized = f.read()

In [ ]:
# ファイルからグラフ定義を取得
import tensorflow as tf

tf.reset_default_graph()
gdef = tf.GraphDef()
gdef.ParseFromString(serialized)

In [ ]:
with tf.Graph().as_default() as g:

    # ファイルから取得したグラフ定義をインポート
    tf.import_graph_def(gdef, name='')
    
    # opのリストを取得
    ops = g.get_operations()
    
    # 最後から3つのopについて出力名を調べる
    N = len(ops)
    for i in [N-3,N-2,N-1]:
        print('\nop id {} : op type: "{}"'.format(str(i), ops[i].type));
        for x in ops[i].outputs:
            print("output name = {}".format(x.name)),


In [ ]:
import tfcoreml

output_tensor_names = ['InceptionV1/Logits/Predictions/Softmax:0']

coreml_model = tfcoreml.convert(
        tf_model_path=tf_model_path,
        mlmodel_path='./inception_v1_simplest.mlmodel',
        output_feature_names=output_tensor_names)

In [ ]:
class_labels = 'imagenet_slim_labels.txt'

coreml_model = tfcoreml.convert(
        tf_model_path=tf_model_path,
        mlmodel_path='./inception_v1_classlabels.mlmodel',
        output_feature_names=output_tensor_names,
        class_labels=class_labels)

In [ ]:
# 最初の3つのopについて出力名を調べる
for i in [0,1,2]:
    print('\nop id {} : op type: "{}"'.format(str(i), ops[i].type));
    for x in ops[i].outputs:
        print("output name = {}".format(x.name)),

In [ ]:
# 入力の型を画像に
image_input_name = ['input:0']

coreml_model = tfcoreml.convert(
        tf_model_path=tf_model_path,
        mlmodel_path='./inception_v1_imageinput.mlmodel',
        image_input_names = image_input_name,
        output_feature_names=output_tensor_names,
        class_labels=class_labels)

In [ ]:
# 前処理を追加
coreml_model = tfcoreml.convert(
        tf_model_path=tf_model_path,
        mlmodel_path='./inception_v1.mlmodel',
        output_feature_names=output_tensor_names,
        image_input_names=image_input_name,
        class_labels=class_labels,
        red_bias = -1,
        green_bias = -1,
        blue_bias = -1,
        image_scale = 2.0/255.0)